# 5. PyTorch Going Modular

This section answers the question, "how do I turn my notebook code into Python scripts?"
## What is going modular?

Turning notebook code into python scripts that offer similar functionality.

For instance:
- `data_setup.py`: file to prepare and downloaded data if needed
- `engine.py`: file with training functions
- `model_builder.py` or `model.py`: file to create PyTorch model
- `train.py`: file to load all needed files to train a PyTorch model
- `utils.py`: file with helpful functions

> **Note:** Naming of the files can be different depending on the cases

## Why would you want to go modular?

**Production code** is code that runs to offer a service to someone or something

It's debatable, but generally one may use Python scripts to provide service, whereas Netflix uses notebooks

### Pros and cons of notebooks vs Python scripts

Notebooks
- Pros
    - Easy to experiment/get started
    - Easy to share
    - Very visual
- Cons
    - Versioning can be hard
    - Hard to use only specific parts
    - Text and graphics can interfere with the code

Python scripts (files ending in `.py`)
- Pros
    - Can package code together (saves rewritting similar code across differnet notebooks)
    - Can use git for versioning
    - Many open source projects use sciprts
    - Larger projects can be run on cloud vendors (not as much support for notebooks)
- Cons
    - Experimenting isn't as visual (usually have to run the whole script rather than one cell)

### My workflow

I think it's best, to begin with a notebook and then move to scripts if needed

### PyTorch in the wild

There are instructions on how to run the code via scripts in code repositories

Below is an example CLI code for training a model and a picture of definitions from https://www.learnpytorch.io/05_pytorch_going_modular/

- `python train.py --model MODEL_NAME --batch_size BATCH_SIZE --lr LEARNING_RATE --num_epochs NUM_EPOCHS`

![image](https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/images/05-python-train-command-line-annotated.png)

## What we're going to cover

Main goal: **turn useful notebook code cells into reusable Python files**

This save us from writing code repeatedly

There are 2 notebooks for this section:
1. [05. Going Modular: Part 1 (cell mode)](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/going_modular/05_pytorch_going_modular_cell_mode.ipynb) - jupyternotebook style file
2. [05. Going Modular: Part 2 (script mode)](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/going_modular/05_pytorch_going_modular_script_mode.ipynb) - Python scripts style file

### Why two parts?

To see side-by-side what's happening and they differ

### What we're working towards

2 main takeaways:
1. Ability to train the model using command line: `python train.py`
2. Directory structure of reusable Python scripts

### Things to note

- **Docstrings**: For reproducibility and understandability, we'll be creating docstrings in each of the functions/classes

- **Imports at the top of scripts**: All scripts require their input modules imported at the start of the script
    - Import all required libraries at the beginning

## 0. Cell mode vs. script mode

## 1. Get data

Data retrieval process is same as in notebook 04

Use `requests` module to download `.zip`file and unzip it

In [ ]:
import os
import requests
import zipfile
from pathlib import Path

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it
if image_path.is_dir():
    (f"{image_path} directory exists")
else:
    print(f"Did not find {image_path} directory, creating one")
    image_path.mkdir(parents=True, exist_ok=True)

# Download data
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading data")
    f.write(request.content)

# Unzip data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unzipping data")
    zip_ref.extractall(image_path)

# Remove zip file
os.remove(data_path / "pizza_steak_sushi.zip")

Unzipping data


## 2. Create Datasets and DataLoaders (`data_setup.py`)

`Dataset` and `DataLoader` code will be created into the `create_dataloaders()` function

We write it to file using the line `%%writefile going_modular/data_setup.py`

> Ensure to create a directory first prior to using `%%writefile` magic command

In [ ]:
# Create the directory if it doesn't exist
os.makedirs("going_modular", exist_ok=True)

In [ ]:
%%writefile going_modular/data_setup.py
"""
Contains functionality for creating PyTorch DataLoaders for
image classification data.
"""
# loading libraries
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
):
    """
    Creates training and testing DataLoaders.

    Takes in a training directory and testing directory path and turns
    them into PyTorch Datasets and then into PyTorch DataLoaders.

    Args:
        train_dir: Path to training directory
        test_dir: Path to testing directory
        transform: torchvision transforms to perform on training and testing data
        batch_size: Number of samples per batch in each of the DataLoaders
        num_workers: An integer for number of workers per DataLoader

    Returns:
        A tuple of (train_dataloader, test_dataloader, class_names)
        Where class_names is a list of the target classes
        Example usage:
            train_dataloader, test_dataloader, class_names = \
                = create_dataloaders(
                    train_dir=path/to/train_dir,
                    test_dir=path/to/test_dir,
                    trasnform=some_transform,
                    batch_size=32,
                    num_workers=4)
    """
    # Use ImageFolder to create dataset(s)
    train_data = datasets.ImageFolder(root=train_dir, transform=transform)
    test_data = datasets.ImageFolder(root=test_dir, transform=transform)

    # Get class class_names
    class_names = train_data.classes

    # Turn images into dataloaders
    train_dataloader = DataLoader(
        dataset=train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )
    
    test_dataloader = DataLoader(
        dataset=test_data,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    return train_dataloader, test_dataloader, class_names

Overwriting going_modular/data_setup.py


We can now use the function within `data_setup.py`
```
# Import data_setup.py
from going_modular import data_setup

# Create train/test dataloader and get class names as a list
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders()
```

## 3. Making a model (`model_builder.py`)

Now it's time to put the model into a script to reuse it.

Let's put `TinyVGG()` model class into a script with the line `%%writefile going_modular/model_builder.py`

In [ ]:
%%writefile going_modular/model_builder.py
"""
Contains PyTorch model code to instantiate a TinyVGG model
"""
import torch
from torch import nn

class TinyVGG(nn.Module):
    """
    Creates the TinyVGG architecture

    Replicates the TinyVGG architecture from the CNN explainer website in PyTorch
    See the original architecture here: https://poloclub.github.io/cnn-explainer/

    Args:
        input_shape: An integer indicating number of input channels
        hidden_units: An integer indicating number of hidden units between layers
        output_shape: An integer indicating number of output units
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2)
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            # Where did this in_features shape come from?
            # It's because each layer of our network compresses and changes the shape of our input data
            nn.Linear(in_features=hidden_units*13*13,
                      out_features=output_shape)
        )
    
    def forward(self, x):
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        return x
        # return self.classifier(self.conv_block_2(self.conv_block_1(x))) # <- leverage the benefits of operator fusion

Overwriting going_modular/model_builder.py


Now we can import the model using
```
import torch
# Import model_builder.py
from going_modular import model_builder
device = "cuda" if torch.cuda.is_available() else "cpu"

# Instantiate an instance of the model from the "model_builder.py" script
# torch.manual_seed(42)
model = model_builder.TinyVGG(input_shape=3,
                              hidden_units=10,
                              output_shape=len(class_names)).to(device)
```

## 4. Creating `train_step()` and `test_step()` functions and `train()` to combine them

We wrote several training functions in notebook 04:
1. `train_step()` - takes in a model, a DataLoader, a loss function and an optimizer and trains the model on the DataLoader.
2. `test_step()` - takes in a model, a DataLoader and a loss function and evaluates the model on the DataLoader.
3. `train()` - performs 1. and 2. together for a given number of epochs and returns a results dictionary.

We can put above functions to `engine.py` with the line `%%writefile going_modular/engine.py`

In [ ]:
%%writefile going_modular/engine.py
"""
Contains functions for training and testing a PyTorch model
"""
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    """
    Trains a PyTorch model for a single epoch

    Turns a target PyTorch model to training mode and then
    runs through all of the required training steps (forward
    pass, loss calculation, optimizer step)

    Args:
        model: A PyTorch model to be trained
        dataloader: A DataLoader instance for the model to be trained on
        loss_fn: A PyTorch loss function to minimize
        optimizer: A PyTorch optimizer to help minimize the loss function
        device: A target device to compute on (e.g. "cuda" or "cpu")

    Returns:
        A tuple of training loss and training accuracy metrics
        in the form (train_loss, train_acc)

        For example: (0.1112, 0.8743)
    """
    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to the target device
        X, y = X.to(device), y.to(device)
        # 1. Forward pass
        y_pred_logits = model(X)
        y_pred_probs = torch.softmax(y_pred_logits, dim=1)
        y_pred = torch.argmax(y_pred_probs, dim=1)

        # 2. Calculate and accumulate loss
        loss = loss_fn(y_pred_logits, y)
        train_loss += loss.item()

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        train_acc += (y_pred==y).sum().item()/len(y_pred_logits)

    # Adjust the metrics to get average loss and accuracy
    train_loss /= len(dataloader)
    train_acc /= len(dataloader)

    return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    """
    Tests a PyTorch model for a single epoch

    Turns a target PyTorch model to "eval" mode and then
    performs a forward pass on a testing dataset

    Args:
        model: A PyTorch model to be tested
        dataloader: A DataLoader instance for the model to be tested on
        loss_fn: A PyTorch loss function to calculate loss on the test data
        device: A target device to compute on (e.g. "cuda" or "cpu")

    Returns:
        A tuple of testing loss and testing accuracy metrics
        in the form (test_loss, test_accuracy)

        For example: (0.0223, 0.8985)
    """
    # Put the model in eval mode
    model.eval()

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on the inference mode context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to the target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)
            test_pred_probs = torch.softmax(test_pred_logits, dim=1)
            test_pred = torch.argmax(test_pred_probs, dim=1)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            test_acc += (test_pred==y).sum().item()/len(test_pred_logits)
        
    # Adjust metrics to get average loss and accuracy
    test_loss /= len(dataloader)
    test_acc /= len(dataloader)

    return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          loss_fn: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
    """
    Trains and tests a PyTorch model

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop

    Calculates, prints and stores evaluation metrics throughout

    Args:
        model: A PyTorch model to be trained and tested
        train_dataloader: A DataLoader instance for the model to be trained on
        test_dataloader: A DataLoader instance for the model to be tested on
        loss_fn: A PyTorch loss function to calculate loss on both datasets
        optimizer: A PyTorch optimizer to help minimize the loss function
        epochs: An integer indicating how many epochs to train for
        device: A target device to compute on (e.g. "cuda" or "cpu")

    Returns:
        A dictionary of training and testing loss as well as accuracy metrics
        Each metric has a value in a list for each epoch

        In the form: {train_loss: [...],
                      train_acc: [...],
                      test_loss: [...],
                      test_acc: [...]}
        For example if training for epochs=2:
                     {train_loss: [2.0616, 1.0537],
                      train_acc: [0.3945, 0.3945],
                      test_loss: [1.2641, 1.5706],
                      test_acc: [0.3400, 0.2973]}
    """
    # Create an empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []}

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer,
                                           device=device)
        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn,
                                        device=device)
        
        # Print out what's happening
        print(f"Epoch: {epoch} | Train loss: {train_loss:.4f} | Train acc: {train_acc:.4f} | Test loss: {test_loss:.4f} | Test acc: {test_acc:.4f}")

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # Return the filled results at the end of the epochs
    return results

Overwriting going_modular/engine.py


Now we've got the `engine.py` script, we can import it via:
```
# Import engine.py
from going_modular import engine

# Use train() by calling it from engine.py
engine.train(...)
```

## 5. Creating a function to save the model (`utils.py`)

It's a common practice to store helper functions in a file called `utils.py` (short for utilities)

Let's save our `save_model()` function to a file called `utils.py` with the line `%%writefile going_modular/utils.py`

In [ ]:
%%writefile going_modular/utils.py
"""
Contains various utility functions for PyTorch model training and saving
"""
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
    """
    Saves a PyTorch model to a target directory

    Args:
        model: A target PyTorch model to save
        target_dir: A directory for saving the model to
        model_name: A filename for the saved model. Should include
            either ".pth" or ".pt" as the file extension
    Example usage:
        save_model(model=model_0,
                   target_dir="models",
                   model_name="05_going_modular_tinyvgg_model.pth")
    """
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                          exist_ok=True)
    
    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt")
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(),
               f=model_save_path)

Overwriting going_modular/utils.py


Now we can save the model via:
```
# Import utils.py
from going_modular import utils

# Save a model to file
save_model(model=model...
           target_dir=...
           model_name=...)
```

## 6. Train, evaluate, and save the model (`train.py`)

Usually you'll find repositories that combine all files put together in `train.py` file

In our `train.py` file, we'll combine all scripts, so we can train a PyTorch model using single line of code:
```
python train.py
```
To create `train.py` we'll go through the following steps:
1. Import libraries and all scripts from `going_modular` directory like `data_setup`, `engine`, `model_builder`, and `utils`
2. **Note**: `train.py` will also be inside going_modular dir, so we can import scripts via `import ...` rather than `from going_modular import ...`
3. Setup hyperparameters like batch size, learning rate, epochs, and hidden units (these could be set in the future via [Python's argpase](https://docs.python.org/3/library/argparse.html))
4. Setup train and test directories
5. Setup device-agnostic code
6. Create necessary data transforms
7. Create DataLoader using `data_setup.py`
8. Create a model using `model_builder.py`
9. Setup loss function and optimizer
10. Train the model using `engine.py`
11. Save the model using `utils.py`

We can create script using the line: `%%writefile going_modular/train.py`

In [ ]:
%%writefile going_modular/train.py
"""
Trains a PyTorch image classification model using device-agnostic code
"""

import os
import torch
import data_setup, engine, model_builder, utils

from torchvision import transforms

# Setup hyperparameters
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

# Setup directories
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.ToTensor()
])

# Create DataLoader using data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

# Create model using model_builder.py
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

# Set loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(),
                             lr=LEARNING_RATE)

# Start training using engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# Save the model using utils.py
utils.save_model(model=model,
                 target_dir="models",
                 model_name="05_going_modular_script_mode_tinyvgg_model.pth")

Overwriting going_modular/train.py


Now we can train a model by running the following line:
```
python train.py
```

And if we wanted to, we could adjust our train.py file to use argument flag inputs with Python's argparse module, this would allow us to provide different hyperparameter settings like previously discussed:
```
python train.py --model MODEL_NAME --batch_size BATCH_SIZE --lr LEARNING_RATE --num_epochs NUM_EPOCHS
```

In [ ]:
!python going_modular/train.py

  0% 0/5 [00:00<?, ?it/s]Epoch: 0 | Train loss: 1.1023 | Train acc: 0.3633 | Test loss: 1.0853 | Test acc: 0.5417
 20% 1/5 [00:01<00:06,  1.53s/it]Epoch: 1 | Train loss: 1.0974 | Train acc: 0.2812 | Test loss: 1.0676 | Test acc: 0.5417
 40% 2/5 [00:03<00:04,  1.51s/it]Epoch: 2 | Train loss: 1.0792 | Train acc: 0.4336 | Test loss: 1.0743 | Test acc: 0.2812
 60% 3/5 [00:05<00:03,  1.93s/it]Epoch: 3 | Train loss: 1.0415 | Train acc: 0.5352 | Test loss: 1.1004 | Test acc: 0.2812
 80% 4/5 [00:06<00:01,  1.75s/it]Epoch: 4 | Train loss: 1.0518 | Train acc: 0.3828 | Test loss: 1.0684 | Test acc: 0.3532
100% 5/5 [00:08<00:00,  1.68s/it]
[INFO] Saving model to: models/05_going_modular_script_mode_tinyvgg_model.pth
